In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [7]:
apikey="sk-proj-WIqONjmhJjlNkeImihiWT3BlbkFJ0za2BbqjSwoZl6PyO7Bk"

prompt=""" Please generate a set of multiple-choice questions (MCQs) with four options each. After generating the questions, provide the output in a dictionary format where each question, its options, and the correct answer can be easily extracted. The dictionary should be structured as follows:

{
  "question_1": {
    "question": "Your question text here?",
    "options": {
      "A": "Option A text",
      "B": "Option B text",
      "C": "Option C text",
      "D": "Option D text"
    },
    "correct_answer": "The correct option letter"
  },
  "question_2": {
    "question": "Your question text here?",
    "options": {
      "A": "Option A text",
      "B": "Option B text",
      "C": "Option C text",
      "D": "Option D text"
    },
    "correct_answer": "The correct option letter"
  },
  ...
}
"""

In [8]:
model = ChatOpenAI(openai_api_key=apikey,model='gpt-4')

In [9]:
from langchain.llms import GooglePalm
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain import HuggingFaceHub
from dotenv import load_dotenv
import os
import pdfkit
from django.http import FileResponse
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import base64
from langchain.document_loaders import PyPDFLoader

import io
import re
from PyPDF2 import PdfReader

from django.http import JsonResponse
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt
import json

In [10]:
prompt=""" Please generate a set of multiple-choice questions (MCQs) with four options each. After generating the questions, provide the output in a dictionary format where each question, its options, and the correct answer can be easily extracted. The dictionary should be structured as follows:

{
  "question_1": {
    "question": "Your question text here?",
    "options": {
      "A": "Option A text",
      "B": "Option B text",
      "C": "Option C text",
      "D": "Option D text"
    },
    "correct_answer": "The correct option letter"
  },
  "question_2": {
    "question": "Your question text here?",
    "options": {
      "A": "Option A text",
      "B": "Option B text",
      "C": "Option C text",
      "D": "Option D text"
    },
    "correct_answer": "The correct option letter"
  },
  ...
}
{context}
"""

In [11]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.output_parsers import OutputFixingParser

# Define the prompt template
prompt_template = PromptTemplate(
    input_variables=["context", "subject", "sub_topic", "num_questions", "difficulty_level", "language", "type"],
    template="""
    Please generate a set of questions based on the following criteria:
    
    - **Subject**: {subject}
    - **Sub-Topic**: {sub_topic}
    - **Number of Questions**: {num_questions}
    - **Difficulty Level**: {difficulty_level}
    - **Language**: {language}
    - **Type**: {type}

    {context}

    ### Instructions:

    Depending on the type specified, create the questions in the appropriate format:
    
    - **MCQs**: Provide multiple-choice questions with four options each.
    - **Question-Answer**: Provide a list of questions with detailed answers.
    - **Fill in the Blanks**: Provide sentences with missing words, indicated by underscores.
     **Matching**: Provide pairs of items to be matched with each other, similar to this example:
      - **Terms**: ["Verbs", "Pronouns", "Adjectives", "Nouns"]
      - **Definitions**: [
          "Words that express actions or states of being",
          "Words that replace nouns",
          "Words that describe or modify nouns",
          "Words that show relationships between nouns and other words"
        ]
    - **Mix**: Create a mixture of the above types, ensuring a variety of question formats.

    ### Output Format:

    For MCQs:
    ```
    {{
      "question_1": {{
        "question": "Your question text here?",
        "options": {{
          "A": "Option A text",
          "B": "Option B text",
          "C": "Option C text",
          "D": "Option D text"
        }},
        "correct_answer": "The correct option letter"
      }},
      ...
    }}
    ```

    For Question-Answer:
    ```
    {{
      "question_1": {{
        "question": "Your question text here?",
        "answer": "Your answer text here"
      }},
      ...
    }}
    ```

    For Fill in the Blanks:
    ```
    {{
      "question_1": {{
        "sentence": "The quick ___ fox jumps over the lazy dog.",
        "answer": "brown"
      }},
      ...
    }}
    ```

     For Matching:
    ```
    {{
      "question_1": {{
        "question defination,
        "definitions": ["Definition 1", "Definition 2", ...],
        "correct_pairs": {{
          "Term 1": "[Definition 1,Defination2]",
          ...
        }}
      }},
      ...
    }}

    For Mix:
    ```
    // A combination of the formats above
    ```

    Please generate the questions accordingly and output them in JSON format.
    """
)

In [12]:
from langchain_core.output_parsers.json import JsonOutputParser

In [13]:
json_parser = JsonOutputParser()

In [18]:
# Initialize the LLM
llm = ChatOpenAI(openai_api_key=apikey,model='gpt-4')

# Define the expected JSON structure for parsing
json_parser = JsonOutputParser()

# Define the output fixing parser with JSON parser and LLM
output_parser = OutputFixingParser.from_llm(parser=json_parser, llm=llm)

mcq_chain = LLMChain(
    prompt=prompt_template,
    llm=llm,
    output_parser=output_parser
)

# Example parameters for generating questions
context = """
Create questions about basic Python programming concepts.
"""
subject = "grammer"
sub_topic = "part of speech"
num_questions = 5
difficulty_level = "Easy"
language = "English"
type = "fill in the blanks"  # Can be MCQs, Question-Answer, Fill in the Blanks, Matching, or Mix

# Run the chain with the specified parameters
result = mcq_chain.run({
    "context": context,
    "subject": subject,
    "sub_topic": sub_topic,
    "num_questions": num_questions,
    "difficulty_level": difficulty_level,
    "language": language,
    "type": type
})


In [19]:
result

{'question_1': {'sentence': 'The ___ is a part of speech that modifies a verb, adjective, or other adverb.',
  'answer': 'adverb'},
 'question_2': {'sentence': 'The ___ is a part of speech that represents a person, place, thing, or idea.',
  'answer': 'noun'},
 'question_3': {'sentence': 'The ___ is a part of speech that replaces a noun in a sentence.',
  'answer': 'pronoun'},
 'question_4': {'sentence': 'The ___ is a part of speech that describes or modifies a noun.',
  'answer': 'adjective'},
 'question_5': {'sentence': 'The ___ is a part of speech that expresses an action or a state of being.',
  'answer': 'verb'}}

In [12]:
for i,j in result.items():
    print(i,"............",j)

question_1 ............ {'question': 'What is the output of the following Python code: print(type(1.0))?', 'options': {'A': "<class 'int'>", 'B': "<class 'str'>", 'C': "<class 'float'>", 'D': "<class 'bool'>"}, 'correct_answer': 'C'}
question_2 ............ {'question': "What is the correct syntax to output 'Hello World' in Python?", 'options': {'A': "echo('Hello World')", 'B': "print('Hello World')", 'C': "output('Hello World')", 'D': "show('Hello World')"}, 'correct_answer': 'B'}
question_3 ............ {'question': 'What will be the output of the following Python code: print(3 * 2 * 1)', 'options': {'A': '2', 'B': '6', 'C': '9', 'D': '1'}, 'correct_answer': 'B'}
question_4 ............ {'question': 'Which of the following is not a valid Python variable name?', 'options': {'A': '_myvar', 'B': 'my_var', 'C': '2myvar', 'D': 'myVar'}, 'correct_answer': 'C'}
question_5 ............ {'question': 'What is the keyword to define a function in Python?', 'options': {'A': 'function', 'B': 'def'

In [ ]:
# Print the JSON output
try:
    output = json.loads(result)
    print(json.dumps(output, indent=2))
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    print("Raw output:", result)


In [28]:
result

{'question_1': {'question': 'What is the output of the following Python code: print(type([]))?',
  'options': {'A': "<class 'list'>",
   'B': "<class 'str'>",
   'C': "<class 'int'>",
   'D': "<class 'float'>"},
  'correct_answer': 'A'},
 'question_2': {'question': 'What is the correct way to comment a line in Python?',
  'options': {'A': '// This is a comment',
   'B': '/* This is a comment */',
   'C': '# This is a comment',
   'D': '-- This is a comment'},
  'correct_answer': 'C'},
 'question_3': {'question': "What is the output of the following Python code: print('Python'[::-1])?",
  'options': {'A': 'Python', 'B': 'nohtyP', 'C': 'nohtyp', 'D': 'No output'},
  'correct_answer': 'B'},
 'question_4': {'question': 'What is the correct syntax to initialize a list in Python?',
  'options': {'A': 'list[] = {}',
   'B': 'list = []',
   'C': 'list{} = []',
   'D': 'list() = []'},
  'correct_answer': 'B'},
 'question_5': {'question': 'Which operator is used for exponentiation in Python?',
 

In [30]:
for question_id, question_details in result.items():
    question_text = question_details['question']
    options = question_details['options']
    correct_answer = question_details['correct_answer']
    
    # Print the extracted information
    print(f"{question_id}: {question_text}")
    for option_key, option_text in options.items():
        print(f"  {option_key}: {option_text}")
    print(f"  Correct Answer: {correct_answer}\n")

question_1: What is the output of the following Python code: print(type([]))?
  A: <class 'list'>
  B: <class 'str'>
  C: <class 'int'>
  D: <class 'float'>
  Correct Answer: A

question_2: What is the correct way to comment a line in Python?
  A: // This is a comment
  B: /* This is a comment */
  C: # This is a comment
  D: -- This is a comment
  Correct Answer: C

question_3: What is the output of the following Python code: print('Python'[::-1])?
  A: Python
  B: nohtyP
  C: nohtyp
  D: No output
  Correct Answer: B

question_4: What is the correct syntax to initialize a list in Python?
  A: list[] = {}
  B: list = []
  C: list{} = []
  D: list() = []
  Correct Answer: B

question_5: Which operator is used for exponentiation in Python?
  A: ^
  B: **
  C: ^^
  D: %%
  Correct Answer: B



In [10]:

text="please generate 12 mcqs on english"

In [12]:
model.invoke(f"{prompt}{text}")

AIMessage(content='{\n  "question_1": {\n    "question": "What is the past tense of \'run\'?",\n    "options": {\n      "A": "Ran",\n      "B": "Runned",\n      "C": "Running",\n      "D": "Runner"\n    },\n    "correct_answer": "A"\n  },\n  "question_2": {\n    "question": "What is a synonym for \'happy\'?",\n    "options": {\n      "A": "Sad",\n      "B": "Angry",\n      "C": "Joyful",\n      "D": "Bored"\n    },\n    "correct_answer": "C"\n  },\n  "question_3": {\n    "question": "What is the plural of \'mouse\'?",\n    "options": {\n      "A": "Mouse",\n      "B": "Mouses",\n      "C": "Mice",\n      "D": "Mousees"\n    },\n    "correct_answer": "C"\n  },\n  "question_4": {\n    "question": "Which one of these is an example of an adverb?",\n    "options": {\n      "A": "Quickly",\n      "B": "Run",\n      "C": "Dog",\n      "D": "Blue"\n    },\n    "correct_answer": "A"\n  },\n  "question_5": {\n    "question": "What is the correct form of \'be\' in present continuous tense?",\n   

In [6]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
prompt=prompt

quiz_generation_prompt = PromptTemplate(
    input_variables=["context"],
    template=prompt
)

In [12]:
mcqschain = LLMChain(llm=model, prompt=quiz_generation_prompt)

C:\Users\PMLS\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [13]:
context_input = "please create the 10 mcqs on english."
result = mcqschain.invoke({"context": context_input})

# Example output handling
quiz_data = result["text"]  # Adjust based on actual output format
print(quiz_data)

ValueError: Missing some input keys: {'\n  "question_1"'}

In [9]:
text="please generate 12 mcqs on english"

In [8]:
TEMPLATE = """
Text:{text}
{prompt}
"""

quizzprompt = PromptTemplate(
    input_variables=["text", "response"],
    template=TEMPLATE
)

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [28]:
chain = create_stuff_documents_chain(model, quizzprompt)
chain.invoke({"text":"ceate the 10 mcqs"})


ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['prompt', 'text']

In [16]:
responce=model.invoke(f"{prompt} {text}")

In [14]:
responce.content

'{\n  "question_1": {\n    "question": "What is the synonym of \'happy\'?",\n    "options": {\n      "A": "Sad",\n      "B": "Elated",\n      "C": "Angry",\n      "D": "Bored"\n    },\n    "correct_answer": "B"\n  },\n  "question_2": {\n    "question": "In the sentence \'The cat is sleeping on the mat\', what is \'mat\'?",\n    "options": {\n      "A": "Subject",\n      "B": "Predicate",\n      "C": "Object",\n      "D": "Verb"\n    },\n    "correct_answer": "C"\n  },\n  "question_3": {\n    "question": "Which word is an adjective in the sentence \'The beautiful garden is full of colorful flowers\'?",\n    "options": {\n      "A": "Beautiful",\n      "B": "Garden",\n      "C": "Full",\n      "D": "Flowers"\n    },\n    "correct_answer": "A"\n  },\n  "question_4": {\n    "question": "What is the plural of \'mouse\'?",\n    "options": {\n      "A": "Mouses",\n      "B": "Mice",\n      "C": "Mouse",\n      "D": "Moose"\n    },\n    "correct_answer": "B"\n  },\n  "question_5": {\n    "ques

In [15]:
print(responce.content)

{
  "question_1": {
    "question": "What is the synonym of 'happy'?",
    "options": {
      "A": "Sad",
      "B": "Elated",
      "C": "Angry",
      "D": "Bored"
    },
    "correct_answer": "B"
  },
  "question_2": {
    "question": "In the sentence 'The cat is sleeping on the mat', what is 'mat'?",
    "options": {
      "A": "Subject",
      "B": "Predicate",
      "C": "Object",
      "D": "Verb"
    },
    "correct_answer": "C"
  },
  "question_3": {
    "question": "Which word is an adjective in the sentence 'The beautiful garden is full of colorful flowers'?",
    "options": {
      "A": "Beautiful",
      "B": "Garden",
      "C": "Full",
      "D": "Flowers"
    },
    "correct_answer": "A"
  },
  "question_4": {
    "question": "What is the plural of 'mouse'?",
    "options": {
      "A": "Mouses",
      "B": "Mice",
      "C": "Mouse",
      "D": "Moose"
    },
    "correct_answer": "B"
  },
  "question_5": {
    "question": "Which of these words is a noun?",
    "option

In [16]:
mcqs=eval(responce.content)

In [17]:
def print_mcqs(mcq_data):
    for key, value in mcq_data.items():
        print(f"Question: {value['question']}")
        for option_key, option_value in value['options'].items():
            print(f"  {option_key}: {option_value}")
        print(f"Correct Answer: {value['correct_answer']}\n")
        
# Print the MCQs
print_mcqs(mcqs)

Question: What is the synonym of 'happy'?
  A: Sad
  B: Elated
  C: Angry
  D: Bored
Correct Answer: B

Question: In the sentence 'The cat is sleeping on the mat', what is 'mat'?
  A: Subject
  B: Predicate
  C: Object
  D: Verb
Correct Answer: C

Question: Which word is an adjective in the sentence 'The beautiful garden is full of colorful flowers'?
  A: Beautiful
  B: Garden
  C: Full
  D: Flowers
Correct Answer: A

Question: What is the plural of 'mouse'?
  A: Mouses
  B: Mice
  C: Mouse
  D: Moose
Correct Answer: B

Question: Which of these words is a noun?
  A: Run
  B: Fast
  C: City
  D: Quickly
Correct Answer: C

Question: What is the past tense of 'run'?
  A: Ran
  B: Runs
  C: Running
  D: Runned
Correct Answer: A

Question: Which of these sentences is grammatically correct?
  A: She don't like coffee.
  B: She doesn't likes coffee.
  C: She doesn't like coffee.
  D: She don't likes coffee.
Correct Answer: C

Question: What is the antonym of 'ancient'?
  A: Old
  B: Modern
  

In [14]:
from langchain.llms import GooglePalm
from langchain import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain import HuggingFaceHub
from dotenv import load_dotenv
import os
import pdfkit
from django.http import FileResponse
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import base64
from langchain.document_loaders import PyPDFLoader

import io
import re
from PyPDF2 import PdfReader

from django.http import JsonResponse
from django.shortcuts import render
from django.views.decorators.csrf import csrf_exempt
import json

In [15]:
def create_pdf_from_text(text, filename):
    # Create a canvas (PDF)
    c = canvas.Canvas(filename, pagesize=letter)

    # Set up the font
    c.setFont("Helvetica", 12)

    # Split the text into lines and draw them on the canvas
    lines = text.split('\n')
    y = 750  # Starting y-coordinate
    for line in lines:
        c.drawString(50, y, line)
        y -= 20  # Move to the next line

    # Save the PDF
    c.save()
    with open("mcqs2.pdf", "rb") as f:
        pdf_content = f.read()
        response = FileResponse(pdf_content, content_type='application/pdf')
        response['Content-Disposition'] = 'inline; filename="mcqs2.pdf"'
        return response

In [8]:
import json

In [9]:
b=responce.content

In [11]:
a=json.dumps(b)

In [16]:
create_pdf_from_text(a,"mcqs2.pdf")

<FileResponse status_code=200, "application/pdf">

In [73]:
def extract_questions(questions_list):
    questions_dict = {}
    current_question = None

    # Iterate through the list
    for item in questions_list:
        # Check if the item starts with '**Question'
        if item.startswith('**'):
            # Extract the question number
            question_number = item.replace('** ', '')
            current_question = question_number
            questions_dict[question_number] = {'question': None, 'question_statement': None, 'options': []}
        elif current_question is not None:
            # Check if the item starts with 'Options'
            if item.startswith('        Options:'):
                # Skip this line
                continue
            else:
                # Check if the item starts with '        a)', '        b)', '        c)', or '        d)'
                if item.startswith('        a)') or item.startswith('        b)') or item.startswith('        c)') or item.startswith('        d)'):
                    # Add the option to the options list
                    questions_dict[current_question]['options'].append(item.strip())
                else:
                    # If it's not an option, it's the question statement
                    questions_dict[current_question]['question_statement'] = item.strip()
        else:
            # If current_question is None, it means we haven't encountered a question yet
            continue

    for question_number, question_info in questions_dict.items():
        print(f"Question {question_number}:")
        print(f"Question: {question_info['question_statement']}")
        print("Options:")
        for option in question_info['options']:
            print(option)
        print()
    return questions_dict


In [74]:
def parse_mcqs(text):
    # Implement your logic to parse the MCQs from the text
    # and return a suitable data structure (e.g., a list of dictionaries)
    # This is just an example, you need to adapt it to your actual MCQ format
    # print(text)
    mcqs = []
    for mcq_text in text.split('\n\n'):
        mcq = {}
        lines = mcq_text.split('\n')
        print(lines)
        dicts=extract_questions(lines)
        print(dicts)
        # mcq['question'] = lines[0]
        # mcq["questionstatement"]=lines[1]
        # print(mcq)
        
        # mcq['options'] = []
        # for option_line in lines[3:]:
        #     if option_line.strip():
        #         option = option_line.split(')')[1].strip()
        #         mcq['options'].append(option)
        # mcqs.append(mcq)
    return dicts

In [75]:
def attempt_quiz():
    # Open the PDF file
    with open("mcqs.pdf", "rb") as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        # print(text)
        
        # Parse the MCQs from the text
    # print("...............................................................................")
    # mcqs = parse_mcqs(text)

    
    return mcqs
    # for question_number, question_info in mcqs.items():
    #     print(f"Question {question_number}:")
    #     print(f"Question: {question_info['question_statement']}")
    #     print("Options:")
    #     for option in question_info['options']:
    #         print(option)
    #     print()
    # return mcqs

In [76]:
a=attempt_quiz()

str

In [77]:
for question_number, question_info in a.items():
        print(f"Question {question_number}:")
        print(f"Question: {question_info['question_statement']}")
        print("Options:")
        for option in question_info['options']:
            print(option)
        print()
   

AttributeError: 'str' object has no attribute 'items'

In [55]:
a

{}

In [25]:
response = """
        
        [Question text here]

        Options:
        a) [Option a text]
        b) [Option b text]
        c) [Option c text]
        d) [Option d text]
"""

text = """
        Quaid-e-Azam Muhammad Ali Jinnah, often referred to as the Father of the Nation in Pakistan, was an exceptional leader whose contributions to the Indian freedom movement and the creation of Pakistan are unparalleled. Born on December 25, 1876, in Karachi, then part of British India, Jinnah's life and legacy continue to inspire millions around the world.
        
        ...
        
        His unwavering commitment to the principles of democracy, tolerance, and equality remains a cornerstone of the nation's identity, reminding Pakistanis of their responsibility to uphold the ideals for which he fought so tirelessly.
        """

TEMPLATE = """
Text:{text}
You are an mcqs generator expert. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} subject and difficult level is {tone}. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like {response} and use it as a guide do not repeat my prompt in your response. \
Ensure to make {number} MCQs do not produce less or greater by your own generate the exact {number} mcqs
"""

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response"],
    template=TEMPLATE
)

mcqschain = LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="mcqs")

TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz.\ check whether it generate {number}  mcqs\
and also check it give the proper response \
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz until whether it matches the exact {number} mcqs which provide in the prompt questions and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{mcqs}

please check the total number of the mcqs it it do not match with the user prompt generate again 

Check from an expert English Writer of the above quiz:
"""
quiz_evaluation_prompt = PromptTemplate(input_variables=['text'], template=TEMPLATE2)

mcqreview = LLMChain(llm=model, prompt=quiz_evaluation_prompt, output_key="reviewmcqs")

generatevaluatechain = SequentialChain(chains=[mcqschain, mcqreview],
                                        input_variables=["text", "number", "subject", "tone", "response"],
                                        output_variables=["mcqs", "reviewmcqs"])

mcq_count = 11
subject = "english"
tone = "difficult"

response = generatevaluatechain(
    {
        "text": text,
    }
)

mcqs = response.get("mcqs")
reviewmcqs = response.get("reviewmcqs")
mcqs = str(mcqs)
create_pdf_from_text(mcqs, "mcqs2.pdf")

with open("mcqs2.pdf", "rb") as f:
    pdf_content = f.read()

C:\Users\PMLS\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\Users\PMLS\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [34]:
response = """
        
        [Question text here]

        Options:
        a) [Option a text]
        b) [Option b text]
        c) [Option c text]
        d) [Option d text]
"""

text = """
        Quaid-e-Azam Muhammad Ali Jinnah, often referred to as the Father of the Nation in Pakistan, was an exceptional leader whose contributions to the Indian freedom movement and the creation of Pakistan are unparalleled. Born on December 25, 1876, in Karachi, then part of British India, Jinnah's life and legacy continue to inspire millions around the world.
        
        ...
        
        His unwavering commitment to the principles of democracy, tolerance, and equality remains a cornerstone of the nation's identity, reminding Pakistanis of their responsibility to uphold the ideals for which he fought so tirelessly.
        """

TEMPLATE = """
Text:{text}
You are an mcqs generator expert. Given the above text, it is your job to \
create a quiz of 6 multiple choice questions for english subject and difficult level is difficult. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like {response} and use it as a guide do not repeat my prompt in your response. \
Ensure to make 6 MCQs do not produce less or greater by your own generate the exact 6 mcqs
"""

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "response"],
    template=TEMPLATE
)

mcqschain = LLMChain(llm=model, prompt=quiz_generation_prompt, output_key="mcqs")

TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for english students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz.\ check whether it generate 6  mcqs\
and also check it give the proper response \
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz until whether it matches the exact 6 mcqs which provide in the prompt questions and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{mcqs}

please check the total number of the mcqs it it do not match with the user prompt generate again 

Check from an expert English Writer of the above quiz:
"""
quiz_evaluation_prompt = PromptTemplate(input_variables=['mcqs'], template=TEMPLATE2)

mcqreview = LLMChain(llm=model, prompt=quiz_evaluation_prompt, output_key="reviewmcqs")

generatevaluatechain = SequentialChain(chains=[mcqschain, mcqreview],
                                        input_variables=["text", "response"],
                                        output_variables=["mcqs", "reviewmcqs"])

response = generatevaluatechain(
    {
        "text": text,
        "response": response
    }
)

mcqs = response.get("mcqs")
reviewmcqs = response.get("reviewmcqs")
mcqs = str(mcqs)
create_pdf_from_text(mcqs, "mcqs.pdf")




<FileResponse status_code=200, "application/pdf">

In [12]:
type(a)

str

In [112]:
print(typeresponce.content)

1. {
    titleofquiz:"Artificial Intelligence Quiz 1",
    question: "What is AI?",
    options: ["a:Artificial Intelligence","b:Applied Intelligence","c:Alphabetical Intelligence","d:Analytical Intelligence"],
    correctanswer:"a:Artificial Intelligence"
}

2. {
    titleofquiz:"Artificial Intelligence Quiz 2",
    question: "Who is known as the father of Artificial Intelligence?",
    options: ["a:Alan Turing","b:John McCarthy","c:Elon Musk","d:Bill Gates"],
    correctanswer:"b:John McCarthy"
}

3. {
    titleofquiz:"Artificial Intelligence Quiz 3",
    question: "What are the subfields of AI?",
    options: ["a:Machine Learning and Deep Learning","b:Robotics and Vision","c:Natural Language Processing and Speech Recognition","d:All of the above"],
    correctanswer:"d:All of the above"
}

4. {
    titleofquiz:"Artificial Intelligence Quiz 4",
    question: "What is the goal of AI?",
    options: ["a:To Create Expert Systems","b:To Implement Human Intelligence in Machines","c:Both a

In [121]:
print(type(json.load(responce.content)))

JSONDecodeError: Extra data: line 1 column 2 (char 1)

In [117]:
a

'"h1. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 1\\",\\n    question: \\"What is AI?\\",\\n    options: [\\"a:Artificial Intelligence\\",\\"b:Applied Intelligence\\",\\"c:Alphabetical Intelligence\\",\\"d:Analytical Intelligence\\"],\\n    correctanswer:\\"a:Artificial Intelligence\\"\\n}\\n\\n2. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 2\\",\\n    question: \\"Who is known as the father of Artificial Intelligence?\\",\\n    options: [\\"a:Alan Turing\\",\\"b:John McCarthy\\",\\"c:Elon Musk\\",\\"d:Bill Gates\\"],\\n    correctanswer:\\"b:John McCarthy\\"\\n}\\n\\n3. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 3\\",\\n    question: \\"What are the subfields of AI?\\",\\n    options: [\\"a:Machine Learning and Deep Learning\\",\\"b:Robotics and Vision\\",\\"c:Natural Language Processing and Speech Recognition\\",\\"d:All of the above\\"],\\n    correctanswer:\\"d:All of the above\\"\\n}\\n\\n4. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 4\\",\\n    

In [102]:
print(type(a))

<class 'str'>


In [85]:
a=responce.content

In [68]:
a.split('\n')[6]

'    question: Which of the following is considered a subfield of artificial intelligence?'

In [103]:
data = {}
for line in a.split('\n'):
    if ':' in line:
        key, value = line.split(':', 1)
        key = key.strip()
        value = value.strip()
        if value.startswith('[') and value.endswith(']'):
            value = json.loads(value)
        data[key] = value

In [108]:
print(data)

{'"1. {\\n    titleofquiz': '\\"Artificial Intelligence Quiz 1\\",\\n    question: \\"What is AI?\\",\\n    options: [\\"a:Artificial Intelligence\\",\\"b:Applied Intelligence\\",\\"c:Alphabetical Intelligence\\",\\"d:Analytical Intelligence\\"],\\n    correctanswer:\\"a:Artificial Intelligence\\"\\n}\\n\\n2. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 2\\",\\n    question: \\"Who is known as the father of Artificial Intelligence?\\",\\n    options: [\\"a:Alan Turing\\",\\"b:John McCarthy\\",\\"c:Elon Musk\\",\\"d:Bill Gates\\"],\\n    correctanswer:\\"b:John McCarthy\\"\\n}\\n\\n3. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 3\\",\\n    question: \\"What are the subfields of AI?\\",\\n    options: [\\"a:Machine Learning and Deep Learning\\",\\"b:Robotics and Vision\\",\\"c:Natural Language Processing and Speech Recognition\\",\\"d:All of the above\\"],\\n    correctanswer:\\"d:All of the above\\"\\n}\\n\\n4. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 4\\",\\n 

{'"1. {\\n    titleofquiz': '\\"Artificial Intelligence Quiz 1\\",\\n    question: \\"What is AI?\\",\\n    options: [\\"a:Artificial Intelligence\\",\\"b:Applied Intelligence\\",\\"c:Alphabetical Intelligence\\",\\"d:Analytical Intelligence\\"],\\n    correctanswer:\\"a:Artificial Intelligence\\"\\n}\\n\\n2. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 2\\",\\n    question: \\"Who is known as the father of Artificial Intelligence?\\",\\n    options: [\\"a:Alan Turing\\",\\"b:John McCarthy\\",\\"c:Elon Musk\\",\\"d:Bill Gates\\"],\\n    correctanswer:\\"b:John McCarthy\\"\\n}\\n\\n3. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 3\\",\\n    question: \\"What are the subfields of AI?\\",\\n    options: [\\"a:Machine Learning and Deep Learning\\",\\"b:Robotics and Vision\\",\\"c:Natural Language Processing and Speech Recognition\\",\\"d:All of the above\\"],\\n    correctanswer:\\"d:All of the above\\"\\n}\\n\\n4. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 4\\",\\n    question: \\"What is the goal of AI?\\",\\n    options: [\\"a:To Create Expert Systems\\",\\"b:To Implement Human Intelligence in Machines\\",\\"c:Both a & b\\",\\"d:None of the above\\"],\\n    correctanswer:\\"c:Both a & b\\"\\n}\\n\\n5. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 5\\",\\n    question: \\"What is Machine Learning?\\",\\n    options: [\\"a:Learning to operate machinery\\",\\"b:Type of AI that allows a machine to learn from experience\\",\\"c:Programming language\\",\\"d:Software application\\"],\\n    correctanswer:\\"b:Type of AI that allows a machine to learn from experience\\"\\n}\\n\\n6. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 6\\",\\n    question: \\"What is the Turing test?\\",\\n    options: [\\"a:Test to measure the intelligence of a machine\\",\\"b:Test to measure the speed of a machine\\",\\"c:Test to measure the power of a machine\\",\\"d:Test to measure the accuracy of a machine\\"],\\n    correctanswer:\\"a:Test to measure the intelligence of a machine\\"\\n}\\n\\n7. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 7\\",\\n    question: \\"Which language is commonly used for AI?\\",\\n    options: [\\"a:Python\\",\\"b:Java\\",\\"c:C++\\",\\"d:All of the above\\"],\\n    correctanswer:\\"d:All of the above\\"\\n}\\n\\n8. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 8\\",\\n    question: \\"What is a neural network?\\",\\n    options: [\\"a:A computer network\\",\\"b:Type of AI algorithm\\",\\"c:Type of computer hardware\\",\\"d:Type of software\\"],\\n    correctanswer:\\"b:Type of AI algorithm\\"\\n}\\n\\n9. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 9\\",\\n    question: \\"What is Deep Learning?\\",\\n    options: [\\"a:A type of Machine Learning\\",\\"b:A programming language\\",\\"c:A computer hardware\\",\\"d:A software application\\"],\\n    correctanswer:\\"a:A type of Machine Learning\\"\\n}\\n\\n10. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 10\\",\\n    question: \\"What is the difference between AI and Machine Learning?\\",\\n    options: [\\"a:AI is a subset of Machine Learning\\",\\"b:Machine Learning is a subset of AI\\",\\"c:AI and Machine Learning are the same\\",\\"d:None of the above\\"],\\n    correctanswer:\\"b:Machine Learning is a subset of AI\\"\\n}\\n\\n11. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 11\\",\\n    question: \\"What is reinforcement learning?\\",\\n    options: [\\"a:Learning from mistakes\\",\\"b:Learning from successes\\",\\"c:Both a & b\\",\\"d:None of the above\\"],\\n    correctanswer:\\"c:Both a & b\\"\\n}\\n\\n12. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 12\\",\\n    question: \\"What is the concept of Natural Language Processing?\\",\\n    options: [\\"a:Understanding and interpreting human language\\",\\"b:Translating human language to machine language\\",\\"c:Programming computers to process large amounts of natural language data\\",\\"d:All of the above\\"],\\n    correctanswer:\\"d:All of the above\\"\\n}\\n\\n13. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 13\\",\\n    question: \\"What is supervised learning?\\",\\n    options: [\\"a:Learning with a teacher\\",\\"b:Learning with a dataset\\",\\"c:Learning with labeled data\\",\\"d:Learning with unlabeled data\\"],\\n    correctanswer:\\"c:Learning with labeled data\\"\\n}\\n\\n14. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 14\\",\\n    question: \\"What is unsupervised learning?\\",\\n    options: [\\"a:Learning without a teacher\\",\\"b:Learning without a dataset\\",\\"c:Learning with labeled data\\",\\"d:Learning with unlabeled data\\"],\\n    correctanswer:\\"d:Learning with unlabeled data\\"\\n}\\n\\n15. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 15\\",\\n    question: \\"What is a Chatbot?\\",\\n    options: [\\"a:An AI program that can engage in conversation\\",\\"b:A type of robot\\",\\"c:A type of software\\",\\"d:A type of programming language\\"],\\n    correctanswer:\\"a:An AI program that can engage in conversation\\"\\n}\\n\\n16. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 16\\",\\n    question: \\"What is computer vision?\\",\\n    options: [\\"a:Ability for a computer to see\\",\\"b:Ability for a computer to interpret visual data\\",\\"c:Both a & b\\",\\"d:None of the above\\"],\\n    correctanswer:\\"c:Both a & b\\"\\n}\\n\\n17. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 17\\",\\n    question: \\"What is a Perceptron?\\",\\n    options: [\\"a:A type of neuron\\",\\"b:A type of robot\\",\\"c:A type of algorithm in machine learning\\",\\"d:A type of computer\\"],\\n    correctanswer:\\"c:A type of algorithm in machine learning\\"\\n}\\n\\n18. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 18\\",\\n    question: \\"What is a Decision Tree?\\",\\n    options: [\\"a:A type of data structure\\",\\"b:A type of machine learning algorithm\\",\\"c:A type of computer hardware\\",\\"d:A type of programming language\\"],\\n    correctanswer:\\"b:A type of machine learning algorithm\\"\\n}\\n\\n19. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 19\\",\\n    question: \\"What is the role of a Data Scientist in AI?\\",\\n    options: [\\"a:Programming AI systems\\",\\"b:Designing AI systems\\",\\"c:Analyzing and interpreting complex digital data\\",\\"d:All of the above\\"],\\n    correctanswer:\\"d:All of the above\\"\\n}\\n\\n20. {\\n    titleofquiz:\\"Artificial Intelligence Quiz 20\\",\\n    question: \\"What is Data Mining?\\",\\n    options: [\\"a:Extracting valuable information from a large dataset\\",\\"b:Storing large amounts of data\\",\\"c:Cleaning large amounts of data\\",\\"d:None of the above\\"],\\n    correctanswer:\\"a:Extracting valuable information from a large dataset\\"\\n}"'}


In [32]:

for i in range(10):
    print(data[f"{i+1}. **Question {i+1}"])

** What is the goal of artificial intelligence (AI)?
** Which programming language is commonly used for AI development?
** What is the Turing Test?
** What is machine learning?
** What is the concept of neural networks in AI?
** Which AI technique involves training a model to perform a specific task by learning from examples?
** What is the primary purpose of natural language processing (NLP) in AI?
** Which AI application is used to recommend products or movies based on a user's past preferences?
** What is the purpose of computer vision in AI?
** What is the concept of reinforcement learning in AI?
